In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from bs4 import BeautifulSoup
import requests
import sys
import os, os.path, csv

# Get List of All 2018 Games

The first step I'd like to take is to create a function to pull every single NFL game for the 2018 season. To do this, I'll be making use of Sportradar's NFL APIs (NFL v5 and NFL Next Generation Stats v1).

In [36]:
#Defining the GET address of the NFL v5 API (with API Key removed.)
url = 'https://api.sportradar.us/nfl/official/trial/v5/en/games/2018/reg/schedule.json?api_key=MY_API_KEY'
response = requests.get(url)

Here I take a look at what this API returns so that I can see what information I'd like to pull and how it's structured.

In [38]:
response.json()

{'_comment': 'Generation started @ 2019-01-18 02:42:02 +0000 ended @ 2019-01-18 02:42:02 +0000',
 'id': '8e45fe2d-fb95-4504-845d-7c815623ccd6',
 'name': 'REG',
 'type': 'REG',
 'weeks': [{'games': [{'attendance': 58699,
     'away': {'alias': 'CIN',
      'game_number': 1,
      'id': 'ad4ae08f-d808-42d5-a1e6-e9bc4e34d123',
      'name': 'Cincinnati Bengals',
      'sr_id': 'sr:competitor:4416'},
     'broadcast': {'network': 'CBS'},
     'entry_mode': 'INGEST',
     'home': {'alias': 'IND',
      'game_number': 1,
      'id': '82cf9565-6eb9-4f01-bdbd-5aa0d472fcd9',
      'name': 'Indianapolis Colts',
      'sr_id': 'sr:competitor:4421'},
     'id': '0822b924-eadc-4398-bfe6-83cbbf3a2912',
     'number': 4,
     'reference': '57570',
     'scheduled': '2018-09-09T17:00:00+00:00',
     'scoring': {'away_points': 34,
      'home_points': 23,
      'periods': [{'away_points': 3,
        'home_points': 3,
        'id': '0e0fc115-8063-490f-8325-37a3442b4a4f',
        'number': 1,
        'pe

'schedule' will be my empty dataframe that I will append to as I iterate through each NFL week to pull the information that I'm interested in from the API response above.

In [68]:
schedule = pd.DataFrame(columns=['Week','Away','Away ID','Away Score',
                                 'Home','Home ID','Home Score',
                                 'Game ID','Weather','Roof Type','Surface'])
schedule

,Week,Away,Away ID,Away Score,Home,Home ID,Home Score,Game ID,Weather,Roof Type,Surface


In [69]:
#For loop that goes through each of the 17 weeks in the NFL season.
for x in range(0,17):
    
    #games will be defined as the number of games played in a given week 'x'.
    #week is the actual week # (ex. x=0 means Week 1)
    games = len(response.json()['weeks'][x]['games'])
    week = x+1
    
    #For loop that goes through each game in a given week.
    for y in range(0,games):
        
        away = response.json()['weeks'][x]['games'][y]['away']['alias']
        away_id = response.json()['weeks'][x]['games'][y]['away']['id']
        away_score = response.json()['weeks'][x]['games'][y]['scoring']['away_points']
        home = response.json()['weeks'][x]['games'][y]['home']['alias']
        home_id = response.json()['weeks'][x]['games'][y]['home']['id']
        home_score = response.json()['weeks'][x]['games'][y]['scoring']['home_points']
        game_id = response.json()['weeks'][x]['games'][y]['id']
        weather = response.json()['weeks'][x]['games'][y]['weather']
        stadium = response.json()['weeks'][x]['games'][y]['venue']['roof_type']
        surface = response.json()['weeks'][x]['games'][y]['venue']['surface']
        
        schedule = schedule.append({'Week':week, 'Away':away, 'Away ID':away_id, 'Away Score':away_score,
                             'Home':home, 'Home ID':home_id, 'Home Score':home_score,
                             'Game ID':game_id, 'Weather':weather, 'Roof Type':stadium, 'Surface':surface},
                            ignore_index=True)
        
schedule.set_index(keys='Game ID',drop=True,inplace=True)

In [73]:
schedule

,Week,Away,Away ID,Away Score,Home,Home ID,Home Score,Weather,Roof Type,Surface
Game ID,,,,,,,,,,
0822b924-eadc-4398-bfe6-83cbbf3a2912,1,CIN,ad4ae08f-d808-42d5-a1e6-e9bc4e34d123,34,IND,82cf9565-6eb9-4f01-bdbd-5aa0d472fcd9,23,"Cloudy Temp: 59 F, Humidity: 91%, Wind: NE 11 mph",retractable_dome,artificial
0a456149-c547-4856-9b1b-86e1d93887ae,1,TB,4254d319-1bc7-4f81-b4ab-b5e6f3402b69,48,NO,0d855753-ea21-4953-89f9-0e20aff9eb73,40,"Partly Cloudy Temp: 86 F, Humidity: 67%, Wind:...",dome,artificial
63047598-04c4-4b94-9674-94beead0724e,1,TEN,d26a1ca5-722d-4274-8f97-c92e49c96315,20,MIA,4809ecb0-abd3-451d-9c4a-92a90b83ca06,27,"Scattered Showers Temp: 89 F, Humidity: 63%, W...",outdoor,turf
8a46621a-752e-4e68-9446-7f6686967064,1,LA,2eff2a03-54d4-46ba-890e-2bc3925548f3,33,OAK,1c1cec48-6352-4556-b789-35304c1a6ae1,13,"Clear Temp: 69 F, Humidity: 58%, Wind: WSW 10 mph",outdoor,turf
8ac7b300-8f92-4a20-a357-4710fd3be679,1,WAS,22052ff7-c065-42ee-bc8f-c4691c50e624,24,ARI,de760528-1dc0-416a-a978-b510d20692ff,6,"N/A Indoor Temp: F, Wind: mph",retractable_dome,turf
8e70d260-ec90-447e-8fdd-10572742d742,1,SF,f0e724b0-4cbf-495a-be47-013907608da9,16,MIN,33405046-04ee-4058-a950-d606f8c30852,24,"N/A (Indoors) Temp: F, Wind: mph",dome,artificial
8f19fa37-ab1a-4624-935e-297d74e20eb8,1,SEA,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,24,DEN,ce92bd47-93d5-4fe9-ada4-0fc681e6caa0,27,"Sunny Temp: 84 F, Humidity: 17%, Wind: Northea...",outdoor,turf
9b12fe52-4398-4976-92e8-35055e996afc,1,JAC,f7ddd7fa-0bae-4f90-bc8e-669e4d6cf2de,20,NYG,04aa1c9d-66da-489d-b16a-1dee3f2eec4d,15,"Rain Temp: 59 F, Humidity: 85%, Wind: NE 7 mph",outdoor,artificial
afec5a57-23c7-4900-bdba-aed594d500a5,1,CHI,7b112545-38e6-483c-a55c-96cf6ee49cb8,23,GB,a20471b4-a8d9-40c7-95ad-90cc30e46932,24,"Clear Temp: 62 F, Humidity: 65%, Wind: E 10 mph",outdoor,turf


I'm happy with this dataframe, so I'll go ahead and save it to a csv so that I can quickly use it later without making the API call again. Below that is where I will start to bypass the API call by reading in the created csv.

In [71]:
schedule.to_csv('2018 NFL Schedule.csv')

In [4]:
schedule = pd.read_csv('2018 NFL Schedule.csv', index_col='Game ID')

# Get List of Los Angeles Rams Plays

Since I'm interested in taking a look at Jared Goff's season, I'll first find the list of Game IDs of all LA Rams game for the 2018 season.

In [5]:
rams_games = list(schedule[(schedule['Away'] == 'LA') | (schedule['Home'] == 'LA')].index)

In [6]:
rams_games

['8a46621a-752e-4e68-9446-7f6686967064',
 'e9624f6c-179d-49d9-8c27-1afe483811a5',
 '79f689e4-4243-4504-b82c-3ed06ccde9d7',
 '5f98a459-16cb-426c-bb73-aef55f2b8c72',
 '7f4745d5-6139-4871-872a-9555d9a455d4',
 'a7d11bde-1b85-4d23-a923-dd06977c1d08',
 '867239c3-7d27-4b04-9955-f9dccc8b15d4',
 '1afbd75e-a90b-46e1-9ba3-08e129073793',
 'ab8482f6-6ad3-45e4-8717-85c1f0c1495f',
 'e97822e2-665f-4118-8911-db18f83e8cd8',
 '6450151d-6068-41b6-a755-cde25323741c',
 '2e27bb65-d582-4a90-9de1-657acf9d1841',
 '70cb6f20-2400-4323-a63b-82ac7fe255a0',
 'd42b4bda-26a5-436b-96e5-3f1b16a244cd',
 'a2b2ed1e-f9a4-4c42-891c-535c844eb63a',
 '83f53db1-4fa9-43c6-8d84-bb142d0feff6']

'rams_plays2018' is my empty dataframe that I'll append to as I find the play-by-play of each Rams game.

In [7]:
rams_plays2018 = pd.DataFrame(columns=['Play ID','Game ID','Description','Team','Quarter','Play Type','Huddle','Clock','Down','Distance','Side','Yard Line',
                             'Snap','Blitz','Rushers','TE Left','TE Right','Box','Direction','Route','Run Direction','Result',
                             'Yards in Air','Yards','Incompletion Type'])

rams_plays2018

,Play ID,Game ID,Description,Team,Quarter,Play Type,Huddle,Clock,Down,Distance,...,TE Left,TE Right,Box,Direction,Route,Run Direction,Result,Yards in Air,Yards,Incompletion Type


In [26]:
#For loop iterates through each of the 16 Rams games.
for game in rams_games:
    
    #I included the print function just as a check to make sure I got through all 16 games.
    print(game)
    
    #API Request with key removed.
    url = 'http://api.sportradar.us/nfl/official/trial/v5/en/games/' + game + '/pbp.json?api_key=MY_API_KEY'
    response = requests.get(url)
    
    #This is the # of periods played in each game, which is fed into the next for loop (ex. 4qtrs, more if OT)
    qtrs = len(response.json()['periods'])
    
    for qtr in range(0,qtrs):
        
        #The response gives us all plays rolled into each possession. So I'll find the # of plays in each poss.
        x = response.json()['periods'][qtr]['pbp']
        possessions = len(x)
        
        #This is the for loop that iterates through each possession based on the # of possessions.
        for poss in range(0,possessions):
            
            #Since not all items are plays, I'll use TRY here to skip over any items that don't have 'events'.
            try:
                plays = len(x[poss]['events'])
            except:
                continue
            
            #Iterate through each play in a given possession.
            for play in range(0,plays):
                
                #Noticed that some plays don't have the info needed, so I'm using individual TRYs here.
                try:
                    y = x[poss]['events'][play]
                except:
                    y = 'N/A'
                
                try:
                    play_type = y['play_type']
                except:
                    play_type = 'N/A'
                
                try:
                    nullified = y['statistics'][0]['nullified']
                except:
                    nullified = 'N/A'
                
                #If the play stood, and was either a pass or rush attempt, get the stats below.
                if nullified == 'N/A' and y != 'N/A' and (play_type == 'pass' or play_type == 'rush'):
                
                    try:
                        huddle = y['huddle']
                    except:
                        huddle = 'N/A'

                    try:
                        blitz = y['blitz']
                    except:
                        blitz = 'N/A'
                    
                    try:
                        te_left = y['left_tightends']
                        te_right= y['right_tightends']
                    except:
                        te_left = 'N/A'
                        te_right = 'N/A'

                    try:
                        box = y['men_in_box']
                    except:
                        box = 'N/A'

                    try:
                        snap = y['qb_at_snap']
                    except:
                        snap = 'N/A'
                    
                    try:
                        direction = y['play_direction']
                    except:
                        direction = 'N/A'

                    try:
                        snap = y['qb_at_snap']
                    except:
                        snap = 'N/A'
                    
                    try:
                        clock = y['clock']
                    except:
                        clock = 'N/A'
                    
                    try:
                        team = y['start_situation']['possession']['alias']
                        down = y['start_situation']['down']
                        distance = y['start_situation']['yfd']
                        side = y['start_situation']['location']['alias']
                        yard_line = y['start_situation']['location']['yardline']
                        description = y['description']
                    except:
                        team = 'N/A'
                        down = 'N/A'
                        distance = 'N/A'
                        side = 'N/A'
                        yard_line = 'N/A'
                        description = 'N/A'
                        
                    play_id = y['id']
                
                    #Stats defined specifically for passing plays.
                    if play_type == 'pass':
                    
                        try:
                            def_rush = y['players_rushed']
                            route = y['pass_route']
                            run_direction = 'N/A'
                            result = y['details'][0]['category']
                            throw_yards = y['statistics'][0]['att_yards']
                        except:
                            def_rush = 'N/A'
                            route = 'N/A'
                            run_direction = 'N/A'
                            result = 'N/A'
                            throw_yards = 'N/A'                           
                    
                        if result == 'pass_completion':
                        
                            try:
                                incompletion = 'N/A'
                                tot_yards = y['statistics'][0]['yards']
                            except:
                                incompletion = 'N/A'
                                tot_yards = 'N/A'                               
                        
                        elif result == 'pass_incompletion':
                        
                            try:
                                incompletion = y['statistics'][0]['incompletion_type']
                                tot_yards = 0
                            except:
                                incompletion = 'N/A'
                                tot_yards = 'N/A'                                
                        
                        else:
                            
                            try:
                                incompletion = 'N/A'
                                tot_yards = 0
                            except:
                                incompletion = 'N/A'
                                tot_yards = 'N/A'                                
                    
                    #Stats defined specifically for rushing attempts.
                    elif play_type == 'rush':
                        
                        try:
                            def_rush = 'N/A'
                            route = 'N/A'
                            run_direction = y['details'][0]['direction']
                            result = 'rush'
                            throw_yards = 'N/A'
                            tot_yards = y['statistics'][0]['yards']    
                            incompletion = 'N/A'
                        except:
                            def_rush = 'N/A'
                            route = 'N/A'
                            run_direction = 'N/A'
                            result = 'N/A'
                            throw_yards = 'N/A'
                            tot_yards = 'N/A'  
                            incompletion = 'N/A'                           
                        
                    else:
                        
                        route = 'N/A'
                        run_direction = 'N/A'
                        result = 'N/A'
                        throw_yards = 'N/A'
                        tot_yards = 'N/A'  
                        incompletion = 'N/A'

                    rams_plays2018 = rams_plays2018.append({'Play ID':play_id,'Game ID':game,'Description':description,'Team':team,'Quarter':qtr+1,'Play Type':play_type,
                              'Clock':clock,'Down':down,'Distance':distance,'Side':side,'Yard Line':yard_line,
                             'Snap':snap,'Blitz':blitz,'Rushers':def_rush,'TE Left':te_left,'TE Right':te_right,'Box':box,
                              'Direction':direction,'Route':route,'Run Direction':run_direction,'Result':result,
                             'Yards in Air':throw_yards,'Yards':tot_yards,'Incompletion Type':incompletion},
                                    ignore_index=True)
                                    
                else:
                    continue
                        
rams_plays2018.set_index(keys='Play ID',drop=True,inplace=True)

8a46621a-752e-4e68-9446-7f6686967064
e9624f6c-179d-49d9-8c27-1afe483811a5
79f689e4-4243-4504-b82c-3ed06ccde9d7
5f98a459-16cb-426c-bb73-aef55f2b8c72
7f4745d5-6139-4871-872a-9555d9a455d4
a7d11bde-1b85-4d23-a923-dd06977c1d08
867239c3-7d27-4b04-9955-f9dccc8b15d4
1afbd75e-a90b-46e1-9ba3-08e129073793
ab8482f6-6ad3-45e4-8717-85c1f0c1495f
e97822e2-665f-4118-8911-db18f83e8cd8
6450151d-6068-41b6-a755-cde25323741c
2e27bb65-d582-4a90-9de1-657acf9d1841
70cb6f20-2400-4323-a63b-82ac7fe255a0
d42b4bda-26a5-436b-96e5-3f1b16a244cd
a2b2ed1e-f9a4-4c42-891c-535c844eb63a
83f53db1-4fa9-43c6-8d84-bb142d0feff6


In [27]:
rams_plays2018

,Game ID,Description,Team,Quarter,Play Type,Huddle,Clock,Down,Distance,Side,...,TE Left,TE Right,Box,Direction,Route,Run Direction,Result,Yards in Air,Yards,Incompletion Type
Play ID,,,,,,,,,,,,,,,,,,,,,
3c8b4474-a1e6-4c12-b57b-fc59a8520f86,8a46621a-752e-4e68-9446-7f6686967064,(15:00) 24-M.Lynch left guard to OAK 28 for 3 ...,OAK,1,rush,NaN,15:00,1,10,OAK,...,1,2,8,Left,N/A,left guard,rush,N/A,3,N/A
d6575d09-e205-42f7-9285-2aa78de3ff57,8a46621a-752e-4e68-9446-7f6686967064,(14:09) 24-M.Lynch up the middle to OAK 35 for...,OAK,1,rush,NaN,14:09,2,2,OAK,...,0,2,7,Middle,N/A,up the middle,rush,N/A,2,N/A
1ffce79e-54a1-45d7-94c7-93adc40a7728,8a46621a-752e-4e68-9446-7f6686967064,(13:25) (Shotgun) 4-D.Carr pass short middle t...,OAK,1,pass,NaN,13:25,1,10,OAK,...,0,0,5,Left,Slant,N/A,pass_completion,8,45,N/A
2f3dd6c8-c40a-4432-901f-7e057d3c475d,8a46621a-752e-4e68-9446-7f6686967064,(12:39) 89-A.Cooper right end pushed ob at LA ...,OAK,1,rush,NaN,12:39,1,10,LA,...,0,2,8,Right Sideline,N/A,right end,rush,N/A,9,N/A
4570981a-8bb9-45b7-86bf-e55f27683eac,8a46621a-752e-4e68-9446-7f6686967064,(12:03) 24-M.Lynch up the middle to LA 7 for 4...,OAK,1,rush,NaN,12:03,2,1,LA,...,0,3,9,Left,N/A,up the middle,rush,N/A,4,N/A
a595a84d-5b04-40e8-8bf3-bb99e22d4d2a,8a46621a-752e-4e68-9446-7f6686967064,(10:58) (Shotgun) 4-D.Carr pass short left to ...,OAK,1,pass,NaN,10:58,1,12,LA,...,0,0,6,Left Sideline,Flat,N/A,pass_completion,1,2,N/A
adee1b5d-24c7-4c0a-bd41-4e3389872d26,8a46621a-752e-4e68-9446-7f6686967064,(10:32) (Shotgun) 24-M.Lynch up the middle for...,OAK,1,rush,NaN,10:32,2,10,LA,...,0,0,6,Middle,N/A,up the middle,rush,N/A,10,N/A
3c6ecd01-9e5f-4ab4-9d31-dcd8b1856b76,8a46621a-752e-4e68-9446-7f6686967064,(10:16) 12-B.Cooks left end to LA 31 for 6 yar...,LA,1,rush,NaN,10:16,1,10,LA,...,0,1,6,Left Sideline,N/A,left end,rush,N/A,6,N/A
ad757076-60b5-409b-a3ca-9ab33b458f4b,8a46621a-752e-4e68-9446-7f6686967064,(9:36) 30-T.Gurley right tackle to LA 33 for 2...,LA,1,rush,NaN,9:36,2,4,LA,...,1,0,6,Right,N/A,right tackle,rush,N/A,2,N/A


Now that I've gotten every play in LA Rams games, I'll filter out their opponents' plays.

In [29]:
rams_plays2018 = rams_plays2018[rams_plays2018['Team'] == 'LA'].copy()

In [30]:
rams_plays2018

,Game ID,Description,Team,Quarter,Play Type,Huddle,Clock,Down,Distance,Side,...,TE Left,TE Right,Box,Direction,Route,Run Direction,Result,Yards in Air,Yards,Incompletion Type
Play ID,,,,,,,,,,,,,,,,,,,,,
3c6ecd01-9e5f-4ab4-9d31-dcd8b1856b76,8a46621a-752e-4e68-9446-7f6686967064,(10:16) 12-B.Cooks left end to LA 31 for 6 yar...,LA,1,rush,NaN,10:16,1,10,LA,...,0,1,6,Left Sideline,N/A,left end,rush,N/A,6,N/A
ad757076-60b5-409b-a3ca-9ab33b458f4b,8a46621a-752e-4e68-9446-7f6686967064,(9:36) 30-T.Gurley right tackle to LA 33 for 2...,LA,1,rush,NaN,9:36,2,4,LA,...,1,0,6,Right,N/A,right tackle,rush,N/A,2,N/A
d3af6bde-3bb9-45cf-be35-4c3f97bf499e,8a46621a-752e-4e68-9446-7f6686967064,(8:52) (Shotgun) 16-J.Goff pass incomplete sho...,LA,1,pass,NaN,8:52,3,2,LA,...,0,0,6,Right Sideline,Curl,N/A,pass_incompletion,6,0,Pass Defended
fcba1473-4c9b-4cc3-99c8-261e72577866,8a46621a-752e-4e68-9446-7f6686967064,(6:07) 16-J.Goff pass incomplete deep left to ...,LA,1,pass,NaN,6:07,1,10,LA,...,0,1,6,Left Sideline,Go,N/A,pass_incompletion,20,0,Dropped Pass
51090770-5075-4f02-b172-b4afc653d90a,8a46621a-752e-4e68-9446-7f6686967064,(6:01) (Shotgun) 16-J.Goff pass short middle t...,LA,1,pass,NaN,6:01,2,10,LA,...,0,0,6,Middle,Curl,N/A,pass_completion,10,11,N/A
55f930e9-9c9b-46c5-b6be-211b057e6a51,8a46621a-752e-4e68-9446-7f6686967064,(5:37) (No Huddle) 16-J.Goff pass deep right t...,LA,1,pass,NaN,5:37,1,10,OAK,...,0,1,6,Right Sideline,Out,N/A,pass_completion,20,20,N/A
b5c83ef0-ceea-4ac1-ba91-5332f7c75a13,8a46621a-752e-4e68-9446-7f6686967064,(4:59) (Shotgun) 16-J.Goff pass short right to...,LA,1,pass,NaN,4:59,1,10,OAK,...,1,0,6,Right Sideline,Underneath Screen,N/A,pass_completion,-4,19,N/A
3b81c229-7b76-4b30-a3ee-4f72c7ece154,8a46621a-752e-4e68-9446-7f6686967064,(12:27) 16-J.Goff pass incomplete short left t...,LA,2,pass,NaN,12:27,1,10,LA,...,0,1,7,Left,Out,N/A,pass_incompletion,4,0,Pass Defended
033877dc-164c-44eb-9f23-74d3b1e73b3c,8a46621a-752e-4e68-9446-7f6686967064,(12:19) 16-J.Goff pass short middle to 17-R.Wo...,LA,2,pass,NaN,12:19,1,10,OAK,...,0,1,6,Left,Slant,N/A,pass_completion,6,10,N/A


Again I'll go ahead and save this to a csv and read it in later.

In [31]:
rams_plays2018.to_csv('Rams 2018.csv')

In [8]:
rams_plays2018 = pd.read_csv('Rams 2018.csv',index_col='Play ID')

Filtering all Rams plays to only show passing plays.

In [114]:
rams_passing = rams_plays2018[rams_plays2018['Play Type'] == 'pass']
rams_passing

,Game ID,Description,Team,Quarter,Play Type,Huddle,Clock,Down,Distance,Side,...,TE Left,TE Right,Box,Direction,Route,Run Direction,Result,Yards in Air,Yards,Incompletion Type
Play ID,,,,,,,,,,,,,,,,,,,,,
d3af6bde-3bb9-45cf-be35-4c3f97bf499e,8a46621a-752e-4e68-9446-7f6686967064,(8:52) (Shotgun) 16-J.Goff pass incomplete sho...,LA,1,pass,NaN,8:52,3,2,LA,...,0,0,6.0,Right Sideline,Curl,NaN,pass_incompletion,6.0,0.0,Pass Defended
fcba1473-4c9b-4cc3-99c8-261e72577866,8a46621a-752e-4e68-9446-7f6686967064,(6:07) 16-J.Goff pass incomplete deep left to ...,LA,1,pass,NaN,6:07,1,10,LA,...,0,1,6.0,Left Sideline,Go,NaN,pass_incompletion,20.0,0.0,Dropped Pass
51090770-5075-4f02-b172-b4afc653d90a,8a46621a-752e-4e68-9446-7f6686967064,(6:01) (Shotgun) 16-J.Goff pass short middle t...,LA,1,pass,NaN,6:01,2,10,LA,...,0,0,6.0,Middle,Curl,NaN,pass_completion,10.0,11.0,NaN
55f930e9-9c9b-46c5-b6be-211b057e6a51,8a46621a-752e-4e68-9446-7f6686967064,(5:37) (No Huddle) 16-J.Goff pass deep right t...,LA,1,pass,NaN,5:37,1,10,OAK,...,0,1,6.0,Right Sideline,Out,NaN,pass_completion,20.0,20.0,NaN
b5c83ef0-ceea-4ac1-ba91-5332f7c75a13,8a46621a-752e-4e68-9446-7f6686967064,(4:59) (Shotgun) 16-J.Goff pass short right to...,LA,1,pass,NaN,4:59,1,10,OAK,...,1,0,6.0,Right Sideline,Underneath Screen,NaN,pass_completion,-4.0,19.0,NaN
3b81c229-7b76-4b30-a3ee-4f72c7ece154,8a46621a-752e-4e68-9446-7f6686967064,(12:27) 16-J.Goff pass incomplete short left t...,LA,2,pass,NaN,12:27,1,10,LA,...,0,1,7.0,Left,Out,NaN,pass_incompletion,4.0,0.0,Pass Defended
033877dc-164c-44eb-9f23-74d3b1e73b3c,8a46621a-752e-4e68-9446-7f6686967064,(12:19) 16-J.Goff pass short middle to 17-R.Wo...,LA,2,pass,NaN,12:19,1,10,OAK,...,0,1,6.0,Left,Slant,NaN,pass_completion,6.0,10.0,NaN
e4efccef-daca-4c12-8170-1a8177b41ab3,8a46621a-752e-4e68-9446-7f6686967064,(10:35) (Shotgun) 16-J.Goff pass incomplete sh...,LA,2,pass,NaN,10:35,2,8,OAK,...,1,0,6.0,Right Sideline,Flat,NaN,pass_incompletion,3.0,0.0,Poorly Thrown
c67b96ca-76ef-48cb-9112-3c747f7f7fcf,8a46621a-752e-4e68-9446-7f6686967064,(10:31) (Shotgun) 16-J.Goff sacked at OAK 29 f...,LA,2,pass,NaN,10:31,3,8,OAK,...,0,0,5.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN


Time to add in Next Generation Stats. First I'll create a function to extract the target out of the 'Description' field

In [98]:
def targets(description):

    #I made this a list since each description has multiple names.
    names=[]

    #Splitting each description into individual words.
    for word in description.split(' '):

        #Default setting is that the word is NOT a player name.
        is_name = False
    
        #For each word, I'll look into each character.
        for char in word:
        
            #Players credited with tackles are in parenthesis, so this signals to skip this word.
            if char == '(':
            
                break
            
            #Players is listed by first initial and last name, so a period means it IS a name.
            if char == '.':
            
                is_name = True
        
        #If the word we have is J.Goff, we aren't interested.
        if word == 'J.Goff':
            
            is_name = False
        
        #If we end up with a player name, it's added to a list of possible pass targets.
        if is_name == True:
        
            names.append(word)
            
    return(names)

Empty dataframe to append Next Gen Stats to.

In [9]:
rams_ngs2018 = pd.DataFrame(columns=['Play ID','O Personnel','Formation','D Personnel','High Safety',
                                    'QB','Start X','Start Y','Target','Target Pos','Target X','Target Y'])

In [116]:
#Iterate through each Rams Passing play.
for x in list(rams_passing.index):
    
    #API request for Next Gen Stats. API key removed.
    url = 'https://api.sportradar.us/nfl-ngs-t1/plays/' + x + '/tracked_statistics.json?api_key=MY_API_KEY'
    response = requests.get(url)
    play = response.json()

    #Where some of LA's stats are depends on if they are home or away, so I'll split those out below.
    if play['play_statistics']['away']['alias'] == 'LA':
        o_personnel = play['play_statistics']['away']['personnel']
        formation = play['play_statistics']['away']['formation']
        d_personnel = play['play_statistics']['home']['personnel']
        high_safety = play['play_statistics']['home']['high_safety']
        
    else:
        o_personnel = play['play_statistics']['home']['personnel']
        formation = play['play_statistics']['home']['formation']
        d_personnel = play['play_statistics']['away']['personnel']
        high_safety = play['play_statistics']['away']['high_safety']
    
    #Each play has multiple stages that I'll call sequence.
    for sequence in play['play_statistics']['events']:

        #When the sequence event is a forward pass, find the player whose position is QB and record their name and location.
        if sequence['event'] == 'pass_forward':
            
            for player in sequence['tracking']['offense']:
                
                if player['position'] == 'QB':
                    
                    qb = player['first_name'] + ' ' + player['last_name']
                    qb_x = player['x']
                    qb_y = player['y']

        #When pass arrives, record the target name, first initial, initial length and last name.
        if sequence['event'] == 'pass_arrived':
            
            target = targets(play['play_statistics']['play']['description'])[0]
            target_first_init = target.split('.')[0]
            init_len = len(target_first_init)
            target_last = target.split('.')[1]
            
            #Go through each player on the field until we find a match to the potential target from the target function.
            for player in sequence['tracking']['offense']:
                
                #Comparing first initial and last name from target function to all players on the field to find a match.
                if (player['first_name'][0:init_len] == target_first_init) & (player['last_name'] == target_last):
                    
                    #Once a match is found, get full name, position (WR, RB, TE, etc.) and location.
                    target_name = player['first_name'] + ' ' + player['last_name']
                    target_pos = player['position']
                    target_x = player['x']
                    target_y = player['y']

    rams_ngs2018 = rams_ngs2018.append({'Play ID':x,'O Personnel':o_personnel,'Formation':formation,'D Personnel':d_personnel,
                                        'High Safety':high_safety,'QB':qb,'Start X':qb_x,'Start Y':qb_y,'Target':target_name,
                                        'Target Pos':target_pos,'Target X':target_x,'Target Y':target_y},
                                      ignore_index=True)
    
rams_ngs2018

KeyError: 'formation'

Notice that I got an error because 'formation' didn't exist somewhere along the way. Let's see how far it got.

In [117]:
rams_ngs2018

,Play ID,O Personnel,Formation,D Personnel,High Safety,QB,Start X,Start Y,Target,Target Pos,Target X,Target Y
0,d3af6bde-3bb9-45cf-be35-4c3f97bf499e,1:RB;1:TE;3:WR,SHOTGUN,4:DL;2:LB;5:DB,1,Jared Goff,84,29.88,Brandin Cooks,WR,70.96,48.73
1,fcba1473-4c9b-4cc3-99c8-261e72577866,1:RB;1:TE;3:WR,SINGLEBACK,4:DL;2:LB;5:DB,1,Jared Goff,70.23,25.26,Cooper Kupp,WR,42.85,3.87
2,51090770-5075-4f02-b172-b4afc653d90a,1:RB;1:TE;3:WR,SHOTGUN,4:DL;2:LB;5:DB,1,Jared Goff,68.02,23.55,Cooper Kupp,WR,49.24,28.02
3,55f930e9-9c9b-46c5-b6be-211b057e6a51,1:RB;1:TE;3:WR,SINGLEBACK,4:DL;2:LB;5:DB,1,Jared Goff,57.92,33.42,Robert Woods,WR,31.05,50.80
4,b5c83ef0-ceea-4ac1-ba91-5332f7c75a13,1:RB;1:TE;3:WR,EMPTY,4:DL;2:LB;5:DB,1,Jared Goff,57.92,33.42,Todd Gurley,RB,33.20,28.27
5,3b81c229-7b76-4b30-a3ee-4f72c7ece154,1:RB;1:TE;3:WR,SINGLEBACK,4:DL;2:LB;5:DB,1,Jared Goff,23.74,23.81,Brandin Cooks,WR,34.43,36.35
6,033877dc-164c-44eb-9f23-74d3b1e73b3c,1:RB;1:TE;3:WR,SINGLEBACK,4:DL;2:LB;5:DB,0,Jared Goff,62.04,22.96,Robert Woods,WR,70.97,31.69
7,e4efccef-daca-4c12-8170-1a8177b41ab3,1:RB;1:TE;3:WR,EMPTY,4:DL;2:LB;5:DB,0,Jared Goff,83.12,24.68,Cooper Kupp,WR,92.68,8.02
8,c67b96ca-76ef-48cb-9112-3c747f7f7fcf,1:RB;1:TE;3:WR,SHOTGUN,4:DL;2:LB;5:DB,2,Jared Goff,83.12,24.68,Cooper Kupp,WR,92.68,8.02
9,b2706738-47b7-4e30-921b-e08cabd5465a,1:RB;1:TE;3:WR,SHOTGUN,4:DL;2:LB;5:DB,0,Jared Goff,97.65,31.27,Robert Woods,WR,109.67,41.32


540 is pretty close to all of the plays, so I'm going to re-write the function with TRYs to skip any fields that don't exist in the remaining plays.

In [118]:
rams_ngs2018_540 = rams_ngs2018

Define 'remainder' as all of the plays after the first 540.

In [121]:
remainder = list(rams_passing.index[540:])

In [122]:
remainder_ngs2018 = pd.DataFrame(columns=['Play ID','O Personnel','Formation','D Personnel','High Safety',
                                    'QB','Start X','Start Y','Target','Target Pos','Target X','Target Y'])

In [126]:
for x in remainder:
    
    url = 'https://api.sportradar.us/nfl-ngs-t1/plays/' + x + '/tracked_statistics.json?api_key=5vnyny7twwc5u3a6fxtggfk3'
    response = requests.get(url)
    play = response.json()
    
    result = rams_passing1.loc['d3af6bde-3bb9-45cf-be35-4c3f97bf499e']['Result']
    
    if play['play_statistics']['away']['alias'] == 'LA':
        try:
            o_personnel = play['play_statistics']['away']['personnel']
            formation = play['play_statistics']['away']['formation']
            d_personnel = play['play_statistics']['home']['personnel']
            high_safety = play['play_statistics']['home']['high_safety']
        except:
            o_personnel = 'N/A'
            formation = 'N/A'
            d_personnel = 'N/A'
            high_safety = 'N/A'         
    else:
        try:
            o_personnel = play['play_statistics']['home']['personnel']
            formation = play['play_statistics']['home']['formation']
            d_personnel = play['play_statistics']['away']['personnel']
            high_safety = play['play_statistics']['away']['high_safety']
            
        except:
            o_personnel = 'N/A'
            formation = 'N/A'
            d_personnel = 'N/A'
            high_safety = 'N/A'   
            
    for sequence in play['play_statistics']['events']:

        if sequence['event'] == 'pass_forward':
            
            for player in sequence['tracking']['offense']:
                
                if player['position'] == 'QB':
                    
                    qb = player['first_name'] + ' ' + player['last_name']
                    qb_x = player['x']
                    qb_y = player['y']

        if sequence['event'] == 'pass_arrived':
            
            target = targets(play['play_statistics']['play']['description'])[0]
            target_first_init = target.split('.')[0]
            init_len = len(target_first_init)
            target_last = target.split('.')[1]
            
            for player in sequence['tracking']['offense']:
                
                if (player['first_name'][0:init_len] == target_first_init) & (player['last_name'] == target_last):
                    
                    target_name = player['first_name'] + ' ' + player['last_name']
                    target_pos = player['position']
                    target_x = player['x']
                    target_y = player['y']

    remainder_ngs2018 = remainder_ngs2018.append({'Play ID':x,'O Personnel':o_personnel,'Formation':formation,'D Personnel':d_personnel,
                                        'High Safety':high_safety,'QB':qb,'Start X':qb_x,'Start Y':qb_y,'Target':target_name,
                                        'Target Pos':target_pos,'Target X':target_x,'Target Y':target_y},
                                      ignore_index=True)
    
remainder_ngs2018

,Play ID,O Personnel,Formation,D Personnel,High Safety,QB,Start X,Start Y,Target,Target Pos,Target X,Target Y
0,900f6717-ff36-4c5c-9577-834b2afcda69,N/A,N/A,N/A,N/A,Jared Goff,77.04,23.59,Todd Gurley,RB,80.71,7.59
1,1ab6c686-7912-40be-833f-fb299c8029b0,0:RB;1:TE;4:WR,EMPTY,4:DL;2:LB;5:DB,5,Jared Goff,83.36,24.69,Todd Gurley,RB,80.71,7.59
2,f7e0c850-d7b9-4b7f-9d68-ad674d81a7cf,1:RB;1:TE;3:WR,SHOTGUN,4:DL;2:LB;5:DB,1,Jared Goff,87.22,29.43,Joshua Reynolds,WR,68.95,8.22
3,72f43268-6820-4919-8ee2-a38adc4d6bc4,1:RB;1:TE;3:WR,PISTOL,4:DL;2:LB;5:DB,1,Jared Goff,66.07,32.95,Cortrelle Anderson,RB,58.76,36.61
4,6b2716a0-437b-4e8e-a95e-5654fdab6e07,1:RB;1:TE;3:WR,WILDCAT,4:DL;2:LB;5:DB,1,Jared Goff,63.63,30.01,Gerald Everett,TE,52.04,48.93
5,b6463961-3e56-4344-825c-26ae89201765,1:RB;1:TE;3:WR,SHOTGUN,4:DL;2:LB;5:DB,1,Jared Goff,63.63,30.01,Gerald Everett,TE,52.04,48.93
6,66a626cf-1965-49d6-ac2b-1dfb45a7c262,1:RB;2:TE;2:WR,SINGLEBACK,4:DL;2:LB;5:DB,1,Jared Goff,55.98,29.85,Robert Woods,WR,8.44,41.94
7,a59da74a-117b-449a-92b4-a5b81c9a3729,1:RB;1:TE;3:WR,SHOTGUN,4:DL;2:LB;5:DB,1,Jared Goff,55.83,25.49,Robert Woods,WR,41.17,15.52
8,015e6a47-49e3-4fd1-bcda-83138b55bed9,1:RB;1:TE;3:WR,WILDCAT,4:DL;2:LB;5:DB,1,Jared Goff,48.75,22.81,Robert Woods,WR,33.75,17.18
9,e3936015-e86b-4cfb-9955-3d1c32b7f44f,1:RB;2:TE;2:WR,PISTOL,4:DL;2:LB;5:DB,0,Jared Goff,39.15,30.71,Tyler Higbee,TE,26.62,41.17


We got the full list now. I'll append the remainder to the first 540 and make this the full dataframe.

In [128]:
rams_ngs2018 = rams_ngs2018_540.append(remainder_ngs2018)
rams_ngs2018.set_index(keys='Play ID',inplace=True)
rams_ngs2018

,O Personnel,Formation,D Personnel,High Safety,QB,Start X,Start Y,Target,Target Pos,Target X,Target Y
Play ID,,,,,,,,,,,
d3af6bde-3bb9-45cf-be35-4c3f97bf499e,1:RB;1:TE;3:WR,SHOTGUN,4:DL;2:LB;5:DB,1,Jared Goff,84,29.88,Brandin Cooks,WR,70.96,48.73
fcba1473-4c9b-4cc3-99c8-261e72577866,1:RB;1:TE;3:WR,SINGLEBACK,4:DL;2:LB;5:DB,1,Jared Goff,70.23,25.26,Cooper Kupp,WR,42.85,3.87
51090770-5075-4f02-b172-b4afc653d90a,1:RB;1:TE;3:WR,SHOTGUN,4:DL;2:LB;5:DB,1,Jared Goff,68.02,23.55,Cooper Kupp,WR,49.24,28.02
55f930e9-9c9b-46c5-b6be-211b057e6a51,1:RB;1:TE;3:WR,SINGLEBACK,4:DL;2:LB;5:DB,1,Jared Goff,57.92,33.42,Robert Woods,WR,31.05,50.80
b5c83ef0-ceea-4ac1-ba91-5332f7c75a13,1:RB;1:TE;3:WR,EMPTY,4:DL;2:LB;5:DB,1,Jared Goff,57.92,33.42,Todd Gurley,RB,33.20,28.27
3b81c229-7b76-4b30-a3ee-4f72c7ece154,1:RB;1:TE;3:WR,SINGLEBACK,4:DL;2:LB;5:DB,1,Jared Goff,23.74,23.81,Brandin Cooks,WR,34.43,36.35
033877dc-164c-44eb-9f23-74d3b1e73b3c,1:RB;1:TE;3:WR,SINGLEBACK,4:DL;2:LB;5:DB,0,Jared Goff,62.04,22.96,Robert Woods,WR,70.97,31.69
e4efccef-daca-4c12-8170-1a8177b41ab3,1:RB;1:TE;3:WR,EMPTY,4:DL;2:LB;5:DB,0,Jared Goff,83.12,24.68,Cooper Kupp,WR,92.68,8.02
c67b96ca-76ef-48cb-9112-3c747f7f7fcf,1:RB;1:TE;3:WR,SHOTGUN,4:DL;2:LB;5:DB,2,Jared Goff,83.12,24.68,Cooper Kupp,WR,92.68,8.02


In [137]:
rams_ngs2018.to_csv('Rams NGS 2018.csv')

# Data Clean-Up

In [2]:
rams_ngs2018 = pd.read_csv('Rams NGS 2018.csv',index_col='Play ID')
rams_reg = pd.read_csv('Rams 2018.csv',index_col='Play ID')

I have all of the X,Y coordinates now of the QB as well as the pass target. What I want to do now is adjust the X coordinate to be the distance from the passer (setting this as 0). I'll have to apply absolute value here since the adjustment could be negative depending on what direction the Rams were driving. 

In [133]:
rams_ngs2018['Target X Adj'] = abs(rams_ngs2018['Target X'] - rams_ngs2018['Start X'])
rams_ngs2018['Start X Adj'] = 0
rams_ngs2018

,O Personnel,Formation,D Personnel,High Safety,QB,Start X,Start Y,Target,Target Pos,Target X,Target Y,Target X Adj,Start X Adj
Play ID,,,,,,,,,,,,,
d3af6bde-3bb9-45cf-be35-4c3f97bf499e,1:RB;1:TE;3:WR,SHOTGUN,4:DL;2:LB;5:DB,1,Jared Goff,84,29.88,Brandin Cooks,WR,70.96,48.73,13.04,0
fcba1473-4c9b-4cc3-99c8-261e72577866,1:RB;1:TE;3:WR,SINGLEBACK,4:DL;2:LB;5:DB,1,Jared Goff,70.23,25.26,Cooper Kupp,WR,42.85,3.87,27.38,0
51090770-5075-4f02-b172-b4afc653d90a,1:RB;1:TE;3:WR,SHOTGUN,4:DL;2:LB;5:DB,1,Jared Goff,68.02,23.55,Cooper Kupp,WR,49.24,28.02,18.78,0
55f930e9-9c9b-46c5-b6be-211b057e6a51,1:RB;1:TE;3:WR,SINGLEBACK,4:DL;2:LB;5:DB,1,Jared Goff,57.92,33.42,Robert Woods,WR,31.05,50.80,26.87,0
b5c83ef0-ceea-4ac1-ba91-5332f7c75a13,1:RB;1:TE;3:WR,EMPTY,4:DL;2:LB;5:DB,1,Jared Goff,57.92,33.42,Todd Gurley,RB,33.20,28.27,24.72,0
3b81c229-7b76-4b30-a3ee-4f72c7ece154,1:RB;1:TE;3:WR,SINGLEBACK,4:DL;2:LB;5:DB,1,Jared Goff,23.74,23.81,Brandin Cooks,WR,34.43,36.35,10.69,0
033877dc-164c-44eb-9f23-74d3b1e73b3c,1:RB;1:TE;3:WR,SINGLEBACK,4:DL;2:LB;5:DB,0,Jared Goff,62.04,22.96,Robert Woods,WR,70.97,31.69,8.93,0
e4efccef-daca-4c12-8170-1a8177b41ab3,1:RB;1:TE;3:WR,EMPTY,4:DL;2:LB;5:DB,0,Jared Goff,83.12,24.68,Cooper Kupp,WR,92.68,8.02,9.56,0
c67b96ca-76ef-48cb-9112-3c747f7f7fcf,1:RB;1:TE;3:WR,SHOTGUN,4:DL;2:LB;5:DB,2,Jared Goff,83.12,24.68,Cooper Kupp,WR,92.68,8.02,9.56,0


Drop plays where Jared Goff was not at QB.

In [3]:
rams_ngs2018.drop(rams_ngs2018.loc[rams_ngs2018['QB']!='Jared Goff'].index, inplace=True)
rams_ngs2018

,O Personnel,Formation,D Personnel,High Safety,QB,Start X,Start Y,Target,Target Pos,Target X,Target Y,Target X Adj,Start X Adj
Play ID,,,,,,,,,,,,,
d3af6bde-3bb9-45cf-be35-4c3f97bf499e,1:RB;1:TE;3:WR,SHOTGUN,4:DL;2:LB;5:DB,1.0,Jared Goff,84.00,29.88,Brandin Cooks,WR,70.96,48.73,13.04,0
fcba1473-4c9b-4cc3-99c8-261e72577866,1:RB;1:TE;3:WR,SINGLEBACK,4:DL;2:LB;5:DB,1.0,Jared Goff,70.23,25.26,Cooper Kupp,WR,42.85,3.87,27.38,0
51090770-5075-4f02-b172-b4afc653d90a,1:RB;1:TE;3:WR,SHOTGUN,4:DL;2:LB;5:DB,1.0,Jared Goff,68.02,23.55,Cooper Kupp,WR,49.24,28.02,18.78,0
55f930e9-9c9b-46c5-b6be-211b057e6a51,1:RB;1:TE;3:WR,SINGLEBACK,4:DL;2:LB;5:DB,1.0,Jared Goff,57.92,33.42,Robert Woods,WR,31.05,50.80,26.87,0
b5c83ef0-ceea-4ac1-ba91-5332f7c75a13,1:RB;1:TE;3:WR,EMPTY,4:DL;2:LB;5:DB,1.0,Jared Goff,57.92,33.42,Todd Gurley,RB,33.20,28.27,24.72,0
3b81c229-7b76-4b30-a3ee-4f72c7ece154,1:RB;1:TE;3:WR,SINGLEBACK,4:DL;2:LB;5:DB,1.0,Jared Goff,23.74,23.81,Brandin Cooks,WR,34.43,36.35,10.69,0
033877dc-164c-44eb-9f23-74d3b1e73b3c,1:RB;1:TE;3:WR,SINGLEBACK,4:DL;2:LB;5:DB,0.0,Jared Goff,62.04,22.96,Robert Woods,WR,70.97,31.69,8.93,0
e4efccef-daca-4c12-8170-1a8177b41ab3,1:RB;1:TE;3:WR,EMPTY,4:DL;2:LB;5:DB,0.0,Jared Goff,83.12,24.68,Cooper Kupp,WR,92.68,8.02,9.56,0
c67b96ca-76ef-48cb-9112-3c747f7f7fcf,1:RB;1:TE;3:WR,SHOTGUN,4:DL;2:LB;5:DB,2.0,Jared Goff,83.12,24.68,Cooper Kupp,WR,92.68,8.02,9.56,0


Now I'll use the Play IDs from above to filter the non-NGS dataframe to show only Jared Goff's plays, then combine the 2 into 1.

In [7]:
goff_reg = rams_reg.loc[rams_ngs2018.index]
goff = goff_reg.join(rams_ngs2018)
goff.head()

,Game ID,Description,Team,Quarter,Play Type,Huddle,Clock,Down,Distance,Side,...,High Safety,QB,Start X,Start Y,Target,Target Pos,Target X,Target Y,Target X Adj,Start X Adj
Play ID,,,,,,,,,,,,,,,,,,,,,
d3af6bde-3bb9-45cf-be35-4c3f97bf499e,8a46621a-752e-4e68-9446-7f6686967064,(8:52) (Shotgun) 16-J.Goff pass incomplete sho...,LA,1,pass,NaN,8:52,3,2,LA,...,1.0,Jared Goff,84.00,29.88,Brandin Cooks,WR,70.96,48.73,13.04,0
fcba1473-4c9b-4cc3-99c8-261e72577866,8a46621a-752e-4e68-9446-7f6686967064,(6:07) 16-J.Goff pass incomplete deep left to ...,LA,1,pass,NaN,6:07,1,10,LA,...,1.0,Jared Goff,70.23,25.26,Cooper Kupp,WR,42.85,3.87,27.38,0
51090770-5075-4f02-b172-b4afc653d90a,8a46621a-752e-4e68-9446-7f6686967064,(6:01) (Shotgun) 16-J.Goff pass short middle t...,LA,1,pass,NaN,6:01,2,10,LA,...,1.0,Jared Goff,68.02,23.55,Cooper Kupp,WR,49.24,28.02,18.78,0
55f930e9-9c9b-46c5-b6be-211b057e6a51,8a46621a-752e-4e68-9446-7f6686967064,(5:37) (No Huddle) 16-J.Goff pass deep right t...,LA,1,pass,NaN,5:37,1,10,OAK,...,1.0,Jared Goff,57.92,33.42,Robert Woods,WR,31.05,50.80,26.87,0
b5c83ef0-ceea-4ac1-ba91-5332f7c75a13,8a46621a-752e-4e68-9446-7f6686967064,(4:59) (Shotgun) 16-J.Goff pass short right to...,LA,1,pass,NaN,4:59,1,10,OAK,...,1.0,Jared Goff,57.92,33.42,Todd Gurley,RB,33.20,28.27,24.72,0


In [8]:
goff.info()

<class 'pandas.core.frame.DataFrame'>
Index: 594 entries, d3af6bde-3bb9-45cf-be35-4c3f97bf499e to 0d7e19a8-44d1-4d74-bb55-73156bc4acbf
Data columns (total 37 columns):
Game ID              594 non-null object
Description          594 non-null object
Team                 594 non-null object
Quarter              594 non-null int64
Play Type            594 non-null object
Huddle               0 non-null float64
Clock                594 non-null object
Down                 594 non-null int64
Distance             594 non-null int64
Side                 594 non-null object
Yard Line            594 non-null int64
Snap                 594 non-null object
Blitz                594 non-null bool
Rushers              548 non-null float64
TE Left              594 non-null int64
TE Right             594 non-null int64
Box                  592 non-null float64
Direction            560 non-null object
Route                548 non-null object
Run Direction        0 non-null object
Result               

I want to  update Incompletion Type to have a little bit more info. Below I'll add Interceptions as they do not currently show up, and I'll throw completions in as well. This will basically be a more detailed version of the 'Result' column. I'm also going to drop Huddle and Run Direction.

In [9]:
goff.drop(labels=['Huddle','Run Direction'], axis=1, inplace=True)

In [10]:
for i,play in goff.iterrows():
    
    if play['Result'] == 'pass_completion':
        goff.loc[i,'Incompletion Type'] = 'Completion'
        
    elif play['Result'] == 'pass_interception':
        goff.loc[i,'Incompletion Type'] = 'Interception'

Now I want to add a way to be able to find completion percentages. To do this, I'll simply add a Completion column that takes a 0 or a 1.

In [11]:
for i,play in goff.iterrows():
    
    if play['Result'] == 'pass_completion':
        goff.loc[i,'Completion'] = 1
        
    else:
        goff.loc[i,'Completion'] = 0

In [12]:
goff.head()

,Game ID,Description,Team,Quarter,Play Type,Clock,Down,Distance,Side,Yard Line,...,QB,Start X,Start Y,Target,Target Pos,Target X,Target Y,Target X Adj,Start X Adj,Completion
Play ID,,,,,,,,,,,,,,,,,,,,,
d3af6bde-3bb9-45cf-be35-4c3f97bf499e,8a46621a-752e-4e68-9446-7f6686967064,(8:52) (Shotgun) 16-J.Goff pass incomplete sho...,LA,1,pass,8:52,3,2,LA,33,...,Jared Goff,84.00,29.88,Brandin Cooks,WR,70.96,48.73,13.04,0,0.0
fcba1473-4c9b-4cc3-99c8-261e72577866,8a46621a-752e-4e68-9446-7f6686967064,(6:07) 16-J.Goff pass incomplete deep left to ...,LA,1,pass,6:07,1,10,LA,50,...,Jared Goff,70.23,25.26,Cooper Kupp,WR,42.85,3.87,27.38,0,0.0
51090770-5075-4f02-b172-b4afc653d90a,8a46621a-752e-4e68-9446-7f6686967064,(6:01) (Shotgun) 16-J.Goff pass short middle t...,LA,1,pass,6:01,2,10,LA,50,...,Jared Goff,68.02,23.55,Cooper Kupp,WR,49.24,28.02,18.78,0,1.0
55f930e9-9c9b-46c5-b6be-211b057e6a51,8a46621a-752e-4e68-9446-7f6686967064,(5:37) (No Huddle) 16-J.Goff pass deep right t...,LA,1,pass,5:37,1,10,OAK,39,...,Jared Goff,57.92,33.42,Robert Woods,WR,31.05,50.80,26.87,0,1.0
b5c83ef0-ceea-4ac1-ba91-5332f7c75a13,8a46621a-752e-4e68-9446-7f6686967064,(4:59) (Shotgun) 16-J.Goff pass short right to...,LA,1,pass,4:59,1,10,OAK,19,...,Jared Goff,57.92,33.42,Todd Gurley,RB,33.20,28.27,24.72,0,1.0


Currently I have adjusted the X coordinates, but since I want to eventually create a scatter plot, another problem to consider is that the Y coordinates will need to flip depending on what direction the Rams were driving. I want to plot it left to right, which means throws to Goff's left should be on the top half of the plot and throws to the right on the bottom. 

Since direction of the drive is the same for each unique quarter, I took that approach to figure out which Y coordinates needed to be adjust. First I'll make a Qtr ID.

In [16]:
for i,play in goff.iterrows():
    
    goff.loc[i,'Qtr ID'] = str(play['Game ID']) + '-' + str(play['Quarter'])
    
goff.head()

,Game ID,Description,Team,Quarter,Play Type,Clock,Down,Distance,Side,Yard Line,...,Start X,Start Y,Target,Target Pos,Target X,Target Y,Target X Adj,Start X Adj,Completion,Qtr ID
Play ID,,,,,,,,,,,,,,,,,,,,,
d3af6bde-3bb9-45cf-be35-4c3f97bf499e,8a46621a-752e-4e68-9446-7f6686967064,(8:52) (Shotgun) 16-J.Goff pass incomplete sho...,LA,1,pass,8:52,3,2,LA,33,...,84.00,29.88,Brandin Cooks,WR,70.96,48.73,13.04,0,0.0,8a46621a-752e-4e68-9446-7f6686967064-1
fcba1473-4c9b-4cc3-99c8-261e72577866,8a46621a-752e-4e68-9446-7f6686967064,(6:07) 16-J.Goff pass incomplete deep left to ...,LA,1,pass,6:07,1,10,LA,50,...,70.23,25.26,Cooper Kupp,WR,42.85,3.87,27.38,0,0.0,8a46621a-752e-4e68-9446-7f6686967064-1
51090770-5075-4f02-b172-b4afc653d90a,8a46621a-752e-4e68-9446-7f6686967064,(6:01) (Shotgun) 16-J.Goff pass short middle t...,LA,1,pass,6:01,2,10,LA,50,...,68.02,23.55,Cooper Kupp,WR,49.24,28.02,18.78,0,1.0,8a46621a-752e-4e68-9446-7f6686967064-1
55f930e9-9c9b-46c5-b6be-211b057e6a51,8a46621a-752e-4e68-9446-7f6686967064,(5:37) (No Huddle) 16-J.Goff pass deep right t...,LA,1,pass,5:37,1,10,OAK,39,...,57.92,33.42,Robert Woods,WR,31.05,50.80,26.87,0,1.0,8a46621a-752e-4e68-9446-7f6686967064-1
b5c83ef0-ceea-4ac1-ba91-5332f7c75a13,8a46621a-752e-4e68-9446-7f6686967064,(4:59) (Shotgun) 16-J.Goff pass short right to...,LA,1,pass,4:59,1,10,OAK,19,...,57.92,33.42,Todd Gurley,RB,33.20,28.27,24.72,0,1.0,8a46621a-752e-4e68-9446-7f6686967064-1


Now I'm going to attempt to extract which side Goff threw to from each play's description. This will allow me to compare the side to the target Y coordinate to find out which direction the Rams were driving.

In [17]:
def side_throw(description):
    
    #Loop through each word in the description.
    for word in description.split(' '):

        #Default for side is N/A
        side = 'N/A'
        
        #If the word 'left' is in the description, set side as Left. Same for Right.
        if word == 'left':
            
            side = 'Left'
            break
        
        if word == 'right':
            
            side = 'Right'
            break
            
    return(side)

In [19]:
for i,play in goff.iterrows():
    
    side = side_throw(play['Description'])
    goff.loc[i,'Throw_Side'] = side
    
    if side == 'N/A':
        
        goff.loc[i,'Driving'] = 'N/A'
    
    #If throw is to Goff's left and Y is in the upper half, we know its Left to Right.
    elif (float(play['Target Y']) > 26.65) & (side == 'Left'):
        
        goff.loc[i,'Driving'] = 'Left to Right'        
    
    #If throw is to Goff's Right and Y is in the lower half, we know its Left to Right.
    elif (float(play['Target Y']) < 26.65) & (side == 'Right'):
        
        goff.loc[i,'Driving'] = 'Left to Right' 
    
    #If throw is to Goff's Right and Y is in the upper half, we know its Right to Left.
    elif (float(play['Target Y']) > 26.65) & (side == 'Right'):
        
        goff.loc[i,'Driving'] = 'Right to Left' 
    
    #If throw is to Goff's Left and Y is in the lower half, we know its Right to Left.
    elif (float(play['Target Y']) < 26.65) & (side == 'Left'):
          
        goff.loc[i,'Driving'] = 'Right to Left' 
    
    else:
        goff.loc[i,'Driving'] = 'N/A' 
        
goff.head()

,Game ID,Description,Team,Quarter,Play Type,Clock,Down,Distance,Side,Yard Line,...,Target,Target Pos,Target X,Target Y,Target X Adj,Start X Adj,Completion,Qtr ID,Throw_Side,Driving
Play ID,,,,,,,,,,,,,,,,,,,,,
d3af6bde-3bb9-45cf-be35-4c3f97bf499e,8a46621a-752e-4e68-9446-7f6686967064,(8:52) (Shotgun) 16-J.Goff pass incomplete sho...,LA,1,pass,8:52,3,2,LA,33,...,Brandin Cooks,WR,70.96,48.73,13.04,0,0.0,8a46621a-752e-4e68-9446-7f6686967064-1,Right,Right to Left
fcba1473-4c9b-4cc3-99c8-261e72577866,8a46621a-752e-4e68-9446-7f6686967064,(6:07) 16-J.Goff pass incomplete deep left to ...,LA,1,pass,6:07,1,10,LA,50,...,Cooper Kupp,WR,42.85,3.87,27.38,0,0.0,8a46621a-752e-4e68-9446-7f6686967064-1,Left,Right to Left
51090770-5075-4f02-b172-b4afc653d90a,8a46621a-752e-4e68-9446-7f6686967064,(6:01) (Shotgun) 16-J.Goff pass short middle t...,LA,1,pass,6:01,2,10,LA,50,...,Cooper Kupp,WR,49.24,28.02,18.78,0,1.0,8a46621a-752e-4e68-9446-7f6686967064-1,N/A,N/A
55f930e9-9c9b-46c5-b6be-211b057e6a51,8a46621a-752e-4e68-9446-7f6686967064,(5:37) (No Huddle) 16-J.Goff pass deep right t...,LA,1,pass,5:37,1,10,OAK,39,...,Robert Woods,WR,31.05,50.80,26.87,0,1.0,8a46621a-752e-4e68-9446-7f6686967064-1,Right,Right to Left
b5c83ef0-ceea-4ac1-ba91-5332f7c75a13,8a46621a-752e-4e68-9446-7f6686967064,(4:59) (Shotgun) 16-J.Goff pass short right to...,LA,1,pass,4:59,1,10,OAK,19,...,Todd Gurley,RB,33.20,28.27,24.72,0,1.0,8a46621a-752e-4e68-9446-7f6686967064-1,Right,Right to Left


Notice that some of the 'Driving' labels are N/A for various reasons. This takes me back to those Qtr IDs that I made. Drives are in the same direction per Quarter, so I can fill in the N/A's based on plays that yielded a result. I did find 1 anomoly after going through some, so I'll fill them in based on a majority. (ex. 1 Left to Right and 8 Right to Lefts in a given quarter, safe to assume Right to Left)

In [21]:
quarters = pd.DataFrame(columns=['Qtr ID','Driving'])

#Finds unique Qtr IDs and loops through them.
for qtr in list(goff['Qtr ID'].unique()):
    
    #Identifies which label is the maximum.
    driving = goff[goff['Qtr ID'] == qtr]['Driving'].value_counts().idxmax()
    
    #Adds the Qtr ID and its max occuring value to a new table.
    quarters = quarters.append({'Qtr ID':qtr,'Driving':driving},ignore_index=True)
    
quarters.set_index(keys='Qtr ID',inplace=True)
quarters.head(1)

,Driving
Qtr ID,
8a46621a-752e-4e68-9446-7f6686967064-1,Right to Left


Create a dictionary from this table to be used on the main data frame.

In [22]:
qtrs_dict = quarters.to_dict()['Driving']

In [23]:
goff['Driving'] = goff['Qtr ID'].apply(lambda x: qtrs_dict[x])
goff.head()

,Game ID,Description,Team,Quarter,Play Type,Clock,Down,Distance,Side,Yard Line,...,Target,Target Pos,Target X,Target Y,Target X Adj,Start X Adj,Completion,Qtr ID,Throw_Side,Driving
Play ID,,,,,,,,,,,,,,,,,,,,,
d3af6bde-3bb9-45cf-be35-4c3f97bf499e,8a46621a-752e-4e68-9446-7f6686967064,(8:52) (Shotgun) 16-J.Goff pass incomplete sho...,LA,1,pass,8:52,3,2,LA,33,...,Brandin Cooks,WR,70.96,48.73,13.04,0,0.0,8a46621a-752e-4e68-9446-7f6686967064-1,Right,Right to Left
fcba1473-4c9b-4cc3-99c8-261e72577866,8a46621a-752e-4e68-9446-7f6686967064,(6:07) 16-J.Goff pass incomplete deep left to ...,LA,1,pass,6:07,1,10,LA,50,...,Cooper Kupp,WR,42.85,3.87,27.38,0,0.0,8a46621a-752e-4e68-9446-7f6686967064-1,Left,Right to Left
51090770-5075-4f02-b172-b4afc653d90a,8a46621a-752e-4e68-9446-7f6686967064,(6:01) (Shotgun) 16-J.Goff pass short middle t...,LA,1,pass,6:01,2,10,LA,50,...,Cooper Kupp,WR,49.24,28.02,18.78,0,1.0,8a46621a-752e-4e68-9446-7f6686967064-1,N/A,Right to Left
55f930e9-9c9b-46c5-b6be-211b057e6a51,8a46621a-752e-4e68-9446-7f6686967064,(5:37) (No Huddle) 16-J.Goff pass deep right t...,LA,1,pass,5:37,1,10,OAK,39,...,Robert Woods,WR,31.05,50.80,26.87,0,1.0,8a46621a-752e-4e68-9446-7f6686967064-1,Right,Right to Left
b5c83ef0-ceea-4ac1-ba91-5332f7c75a13,8a46621a-752e-4e68-9446-7f6686967064,(4:59) (Shotgun) 16-J.Goff pass short right to...,LA,1,pass,4:59,1,10,OAK,19,...,Todd Gurley,RB,33.20,28.27,24.72,0,1.0,8a46621a-752e-4e68-9446-7f6686967064-1,Right,Right to Left


Now I can adjust all of the Y coordinates based on the drive information. Remember, the width of the field if 53-1/3 yards.

In [24]:
for i,play in goff.iterrows():
    
    if play['Driving'] == 'Right to Left':
        goff.loc[i,'Target Y Adj'] = 53.3 - play['Target Y']
        goff.loc[i,'Start Y Adj'] = 53.3 - play['Start Y']
        
    else:
        goff.loc[i,'Target Y Adj'] = play['Target Y']
        goff.loc[i,'Start Y Adj'] = play['Start Y']   

Finally, I want to group all of the X coordinates and Y coordinates.

In [25]:
for i,play in goff.iterrows():
    
    if play['Target X Adj'] <= 5:
        goff.loc[i,'X Group'] = '0-5 Yds'
        
    elif play['Target X Adj'] <= 10:
        goff.loc[i,'X Group'] = '5-10 Yds'

    elif play['Target X Adj'] <= 15:
        goff.loc[i,'X Group'] = '10-15 Yds'
        
    elif play['Target X Adj'] <= 20:
        goff.loc[i,'X Group'] = '15-20 Yds'

    elif play['Target X Adj'] <= 25:
        goff.loc[i,'X Group'] = '20-25 Yds'

    elif play['Target X Adj'] <= 30:
        goff.loc[i,'X Group'] = '25-30 Yds'
        
    elif play['Target X Adj'] <= 35:
        goff.loc[i,'X Group'] = '30-35 Yds'

    elif play['Target X Adj'] <= 40:
        goff.loc[i,'X Group'] = '35-40 Yds'

    elif play['Target X Adj'] <= 45:
        goff.loc[i,'X Group'] = '40-45 Yds'
        
    elif play['Target X Adj'] <= 50:
        goff.loc[i,'X Group'] = '45-50 Yds'

    elif play['Target X Adj'] <= 55:
        goff.loc[i,'X Group'] = '50-55 Yds'

    elif play['Target X Adj'] <= 60:
        goff.loc[i,'X Group'] = '55-60 Yds'
        
    elif play['Target X Adj'] <= 65:
        goff.loc[i,'X Group'] = '60-65 Yds'
        
    elif play['Target X Adj'] <= 70:
        goff.loc[i,'X Group'] = '65-70 Yds'
        
    elif play['Target X Adj'] <= 75:
        goff.loc[i,'X Group'] = '70-75 Yds'

    elif play['Target X Adj'] <= 80:
        goff.loc[i,'X Group'] = '75-80 Yds'

    elif play['Target X Adj'] <= 85:
        goff.loc[i,'X Group'] = '80-85 Yds'
        
    elif play['Target X Adj'] <= 90:
        goff.loc[i,'X Group'] = '85-90 Yds'

    elif play['Target X Adj'] <= 95:
        goff.loc[i,'X Group'] = '90-95 Yds'

    elif play['Target X Adj'] <= 100:
        goff.loc[i,'X Group'] = '95-100 Yds'
        
    elif play['Target X Adj'] > 100:
        goff.loc[i,'X Group'] = '>100 Yds'

In [26]:
for i,play in goff.iterrows():
    
    if play['Target Y Adj'] <= 10.66:
        goff.loc[i,'Y Group'] = 'Right'
        
    elif play['Target Y Adj'] <= 21.32:
        goff.loc[i,'Y Group'] = 'Mid-Right'

    elif play['Target Y Adj'] <= 31.98:
        goff.loc[i,'Y Group'] = 'Middle'
        
    elif play['Target Y Adj'] <= 42.64:
        goff.loc[i,'Y Group'] = 'Mid-Left'

    elif play['Target Y Adj'] <= 53.33:
        goff.loc[i,'Y Group'] = 'Left'

In [28]:
goff.to_csv('Goff 2018 NGS.csv')

That's it! I now have a variety of stats/metrics that I can use for interesting analysis/visualizations which I'll be performing in the Data Exploration notebook.